<a href="https://colab.research.google.com/github/yiruchen1993/tensorflow-practice/blob/master/transfer_learning_dogbreed_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO: 增加訓練資料 (增加到3000會導致Ram不夠)


In [1]:
from google.colab import drive 
# 將 google drive 掛載在 colob
drive.mount('/content/gdrive') 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd 'gdrive/My Drive/study/dogbreed'

/content/gdrive/My Drive/study/dogbreed


In [3]:
import os
import pickle

In [4]:
def pkl_save(pkl_filename, obj):
    with open(pkl_filename, "wb") as file_write:
        pickle.dump(obj, file_write)

def pkl_load(pkl_filename):
    with open(pkl_filename, "rb") as file_read:
        obj = pickle.load(file_read)
        return obj

In [5]:
if not os.path.exists("dogbreed"):
    !unzip -u "dog-breed-identification.zip" -d "dogbreed"
else:
    print("data already exists")

data already exists


In [6]:
%cd 'dogbreed'

/content/gdrive/My Drive/study/dogbreed/dogbreed


In [7]:
import pandas as pd
import cv2
from os import walk
import numpy as np

In [8]:
y_label = pd.read_csv('labels.csv')

In [9]:
if not os.path.exists("train_fig.pickle"):
    train_fig = []
    for (dirpath, dirnames, filenames) in walk("./train/"):
        train_fig.extend(filenames)
        break
else:
    print("train_fig already exists")

if not os.path.exists("test_fig.pickle"):
    test_fig = []
    for (dirpath, dirnames, filenames) in walk("./test/"):
        test_fig.extend(filenames)
        break
else:
    print("test_fig already exists")

train_fig already exists
test_fig already exists


In [10]:
# train_fig = pkl_load("train_fig.pickle")
# test_fig = pkl_load("test_fig.pickle")

In [11]:
width = 300

In [12]:
if not os.path.exists("x_train_raw.pickle"):
    x_train_raw = np.zeros((n_train, width, width, 3), dtype=np.uint8)
    for i in range(n_train):
        print(i)
        image_path = "./train/{}".format(train_fig[i])
        x_train_raw[i] = cv2.resize(cv2.imread(image_path, cv2.IMREAD_COLOR), (width, width))
    pkl_save("x_train_raw.pickle", x_train_raw)
else:
    print("x_train_raw already exists")

x_train_raw already exists


In [13]:
if not os.path.exists("x_test_raw.pickle"):
    x_test_raw = np.zeros((n_test, width, width, 3), dtype=np.uint8)
    for i in range(n_test):
        print(i)
        image_path = "./test/{}".format(test_fig[i])
        x_test_raw[i] = cv2.resize(cv2.imread(image_path, cv2.IMREAD_COLOR), (width, width))
    pkl_save("x_test_raw.pickle", x_test_raw)
else:
    print("x_test_raw already exists")

x_test_raw already exists


In [14]:
# x_train_raw = pkl_load("x_train_raw.pickle")
# x_test_raw = pkl_load("x_test_raw.pickle")

In [15]:
n_train = 2500
n_test = 500

In [16]:
# Normalize Data
def normalize(X_train, X_test):
    mean = np.mean(X_train, axis=(0, 1, 2, 3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train - mean) / (std + 1e-7)
    X_test = (X_test - mean) / (std + 1e-7)
    return X_train, X_test


In [17]:
def get_training_data(is_retrain=False, n_train=3000, n_test=300):
    if is_retrain:
        x_train_raw = pkl_load("x_train_raw.pickle")
        # x_test_raw = pkl_load("x_test_raw.pickle")

        os.remove("x_train_raw_part.pickle")
        os.remove("x_test_raw_part.pickle")

        x_train_raw_part = x_train_raw[0:n_train, :, :, :]
        pkl_save("x_train_raw_part.pickle", x_train_raw_part)
        x_test_raw_part = x_train_raw[n_train:(n_train+n_test), :, :, :]
        pkl_save("x_test_raw_part.pickle", x_test_raw_part)

        del x_train_raw
    else:
        x_train_raw_part = pkl_load("x_train_raw_part.pickle")
        x_test_raw_part = pkl_load("x_test_raw_part.pickle")
    
    # Normalize Training and Testset
    x_train, x_test = normalize(x_train_raw_part, x_test_raw_part)
    del x_train_raw_part
    del x_test_raw_part

    return x_train, x_test

In [18]:
x_train, x_test = get_training_data(is_retrain=True, n_train=n_train, n_test=n_test)

In [19]:
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

In [20]:
# y_train = np.array(y_label.breed)
y_label_all = np.array(y_label.breed)

# One-hot
labelen = preprocessing.LabelEncoder()
y_label_all = labelen.fit_transform(y_label_all)
onehotencoder = OneHotEncoder()
y_label_all_onehot = onehotencoder.fit_transform(y_label_all.reshape(-1, 1)).toarray()

In [21]:
y_train = y_label_all_onehot[0:n_train]
y_test = y_label_all[n_train:(n_train+n_test)]

In [22]:
from keras.models import Sequential, Model
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Input
from keras.preprocessing import image
from keras import backend as K
import keras
import tensorflow as tf
import shutil

In [24]:
def get_trained_model(x_train, y_train, is_retrain=False):
    if is_retrain:
        if os.path.exists("trained_model"):
            shutil.rmtree("trained_model")
        os.mkdir("trained_model")

        input_tensor = Input(shape=(32, 32, 3))
        
        # include top 決定要不要加入 fully Connected Layer
        """Resnet 50 架構"""
        model = keras.applications.ResNet50(
            input_shape=(32, 32, 3),
            include_top=False,
            weights="imagenet",
            pooling=None,
            classes=120,
        )
        # model.summary()

        # 添加層數
        x = model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(units=128, activation="relu")(x)
        x = Dropout(rate=0.1)(x)
        predictions = Dense(units=120, activation="softmax")(x)
        model = Model(inputs=model.input, outputs=predictions)
        print("Model depth：", len(model.layers))

        # 鎖定特定幾層不要更新權重
        for layer in model.layers[:100]:
            layer.trainable = False
        for layer in model.layers[100:]:
            layer.trainable = True

        model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        model.fit(x_train, y_train, batch_size=32, epochs=20)

        model.save("trained_model")    
    else:
        model = keras.models.load_model("trained_model")
    
    print(model)
    return model

In [25]:
model = get_trained_model(x_train, y_train, is_retrain=True)

Model depth： 179
Epoch 1/20
79/79 [==============================] - 22s 283ms/step - loss: 4.8102 - accuracy: 0.0056
Epoch 2/20
79/79 [==============================] - 22s 278ms/step - loss: 4.7859 - accuracy: 0.0120
Epoch 3/20
79/79 [==============================] - 22s 280ms/step - loss: 4.7842 - accuracy: 0.0148
Epoch 4/20
79/79 [==============================] - 23s 285ms/step - loss: 4.7823 - accuracy: 0.0140
Epoch 5/20
79/79 [==============================] - 23s 289ms/step - loss: 4.7806 - accuracy: 0.0140
Epoch 6/20
79/79 [==============================] - 23s 292ms/step - loss: 4.7793 - accuracy: 0.0120
Epoch 7/20
79/79 [==============================] - 23s 294ms/step - loss: 4.7763 - accuracy: 0.0156
Epoch 8/20
79/79 [==============================] - 23s 297ms/step - loss: 4.7711 - accuracy: 0.0128
Epoch 9/20
79/79 [==============================] - 24s 299ms/step - loss: 4.7652 - accuracy: 0.0152
Epoch 10/20
79/79 [==============================] - 24s 299ms/step - loss

In [26]:
y_pred = model.predict(x_test)

In [27]:
y_pred_label = np.argmax(y_pred, axis=1)

In [31]:
sum(y_test == y_pred_label)/n_test

0.01